# kappa系数简介

### kappa系数是什么

- Kappa系数用于一致性检验，也可以用于衡量分类精度，kappa系数的计算是基于混淆矩阵的。  
> 分类问题中，这个一致性就是模型预测结果和实际分类结果的一致性
- kappa系数是衡量分类精度的指标。
- kappa系数取值为-1到1之间,通常大于0  
- <font size=4> $kappa = \frac{p_o-p_e}{1-p_e}$ </font>  
> - $p_o$ 等价于acc  
>  $p_o$ = 对角线之和 / 整个矩阵元素之和 
> - $p_e$ 是所有类别对应的“实际与预测数量的乘积之总和”除以“样本总数的平方”  
>  $p_e = \frac{\sum{第i行之和 * 第i列之和}}{矩阵所有元素之和} $


- kappa系数计算示例  


| <font size=4>成绩评级</font> | <font size=4>好</font> | <font size=4>中</font> | <font size=4>差</font> |
| :-: | :-: | :-: | :-: |
| <font size=4>好</font> | <font size=4>a</font> | <font size=4>b</font> | <font size=4>c</font> |
| <font size=4>中</font> | <font size=4>d</font> | <font size=4>e</font> | <font size=4>f</font> |
| <font size=4>差</font> | <font size=4>g</font> | <font size=4>h</font> | <font size=4>i</font> |  
<font size=4>$p_o = \frac{a+e+i}{\sum{a\cdots i}}$</font>  
<font size=4>$p_e = \frac{(a+d+g)\times(a+b+c) + (b+e+h)\times(d+e+f) + (c+f+i)\times(g+h+i)}{(a+b+c+d+e+f+g+h+i)^2}$</font>  

### 为什么要使用kappa
在分类问题中，各个类别的样本数量往往不太平衡，这时模型就容易偏向样本多的类别。此时可能出现整体acc挺高，但是部分类别完全不能被预测的现象。

- eg: 类别0有10个样本，类别1有90的样本，则将所有类别都预测为1，acc也有90%。但类别0就完全被“抛弃”了。 

这时需要一种能够抑制模型的“偏向性”的指标来代替acc。而kappa正好能够抑制模型的“偏向性”。

# 编程计算kappa值

### 基于混淆矩阵，使用nunpy手动实现

In [6]:
import numpy as np

# 没有对输入的合法性进行校验
# 使用时需要注意
def kappa(confusion_matrix):
    """计算kappa值系数"""
    pe_rows = np.sum(confusion_matrix, axis=0)
    pe_cols = np.sum(confusion_matrix, axis=1)
    sum_total = sum(pe_cols)
    pe = np.dot(pe_rows, pe_cols) / float(sum_total ** 2)
    po = np.trace(confusion_matrix) / float(sum_total)
    return (po - pe) / (1 - pe)

- #### 使用无偏向和有偏向的混淆矩阵分别测试

In [7]:
# 无偏向的混淆矩阵
balance_matrix = np.array(
    [
        [2,  1,  1],
        [1,  2,  1],
        [1,  1,  2]
    ]
)

# 有偏向的混淆矩阵
unbalance_matrix = np.array(
    [
        [0,  0,  3],
        [0,  0,  3],
        [0,  0,  6]
    ]
)

In [11]:
kappa_balance = kappa(balance_matrix)
print("kappa for balance matrix: %s" % kappa_balance)

kappa_unbalance = kappa(unbalance_matrix)
print("kappa for unbalance matrix: %s" % kappa_unbalance)

kappa for balance matrix: 0.25
kappa for unbalance matrix: 0.0


> **结论:**  上面两个混淆矩阵的acc一样，但是kappa值不同，偏向性强的kappa值低。

### sklearn中的kappa

In [4]:
from sklearn.metrics import cohen_kappa_score
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
kappa_value = cohen_kappa_score(y_true, y_pred)
print("kappa值为 %f" % kappa_value)

kappa值为 0.428571


### tensorflow中的kappa

In [5]:
import tensorflow as tf

y_t = tf.constant(y_true)
y_p = tf.constant(y_pred)
kappa, update = tf.contrib.metrics.cohen_kappa(y_t, y_p, 3)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print(kappa.eval(), update.eval())
    print(kappa.eval(), update.eval())

C:\Users\hzx\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0.0 0.42857142857142855
0.42857142857142855 0.42857142857142855


> - **注意**  
> 如上例结果  
> tensorflow一开始将kappa初始化为0  
> 返回值中“kappa”是运行计算图之前的kappa值，“update”才是运行了计算图后的kappa值。
> 不可像调用sklearn那样直接把返回的“kappa”作为kappa值